In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
import itertools
import matplotlib.pyplot as plt
from  sklearn.metrics import mean_squared_error as mse
from astropy.io import fits
import matplotlib.pyplot as plt
import os, sys
from numba import njit, jit
from math import isnan
from sklearn.preprocessing import MinMaxScaler

In [15]:
folder_lc = "/work/work_teamEXOPLANET/KOI_LC/"
#Clean Light Curves
lc_kepler = np.load(folder_lc+"npy/KOI_LC_init.npy")
lc_kepler_times = np.load(folder_lc+'npy/KOI_LC_time.npy')

In [16]:
total_time_list = []
for lc,times in zip(lc_kepler,lc_kepler_times):
    time_list = []
    aux_t = times[0]
    for i, data in enumerate(lc):
        if not isnan(data):
            time_list.append(times[i] - aux_t)
        else:
            time_list.append(np.nan)
            #time_list.append(0)
            if (times[i] != times[-1]):
                aux_t = times[i+1]
    total_time_list.append(time_list)

In [ ]:
indice = 14
plt.figure(figsize=(30,10))
plt.plot(total_time_list[indice],'bo')
plt.show()

## Valores entre -1 y 1

In [18]:
## no python: njit -- no soporta lista de lista, lista de arrays, si soporta: lista de sets

########### no lo logre hacer funcionar
#@njit(parallel=True, cache=False, fastmath=True)
def extract_nans(fluxs, plot=True):
    ###extrae los nans y deja tramos de valores continuos
    lc_wind_nan = []
    lengths = []

    sublist = [] #para que sepa que el tipo es float
    for value in fluxs:
        if np.isnan(value) and len(sublist) != 0:
            if len(sublist) != 1: ##borrar tramos de largo 1??
                lc_wind_nan.append(np.asarray(sublist))
                lengths.append(len(sublist))
            sublist = [] #para que sepa que el tipo es float
        elif np.isnan(value) and len(sublist) == 0:
            continue
        else: #if value not nan
            sublist.append(value) 

    if plot:
        lengths = np.asarray(lengths)   
        #print("Cantidad de tramos: ",len(lengths))
        #print("Largo promedio de tramos: ", np.mean(lengths))
        #print("Mediana de largo de tramos: ", np.median(lengths))
        #print("Min de largo de tramos: ", np.min(lengths))
        #print("Max de largo de tramos: ", np.max(lengths))
    return lc_wind_nan


def prepare_lc(fluxs): #dividir por max
    fluxs = np.asarray(fluxs)
    fluxs = fluxs.reshape(-1,1)
    scaler = MinMaxScaler((-1, 1))
    scaler.fit(fluxs)
    fluxs = scaler.transform(fluxs)
    fluxs = fluxs.ravel()
    return fluxs#fluxs/np.abs(np.nanmin(fluxs))

@njit(parallel=True, cache=False, fastmath=True)
def det_state(a, b, n_sta):
    topes=np.linspace(a, b, n_sta+1)
    estados= []
    ind=1
    for top in topes[:-1]:
        estados.append((top,topes[ind]))
        ind+=1
    return estados

@njit(parallel=False, cache=True, fastmath=True)
def det_state_2ways(a, b, n_sta_up, n_sta_low):
    estados_up = det_state(a, 0, n_sta_up)
    estados_low = det_state(0, b, n_sta_low)
    return estados_up + estados_low

@njit(parallel=False, cache=True, fastmath=True)
def det_celda(num, estados): 
    for celda, (est_low, est_up) in enumerate(estados):
        if num<est_low and num>=est_up:
            return celda
    return 0 #if sale por arriba (mayor a 1)

@njit(parallel=False, cache=True, fastmath=True)  #no se pudo paralelizar por el acceso a lista "ind -1"
def add_transitions(fluxs, transition_m, states):
    for ind in range(1,len(fluxs)):
        init_s = det_celda(fluxs[ind-1], states)
        fin_s = det_celda(fluxs[ind], states)
        transition_m[init_s, fin_s] += 1
    return transition_m

@njit(parallel=True, cache=False, fastmath=True)
def manual_HMM(wind_fluxs, n_sta_up, n_sta_low=0):
    if n_sta_low == 0: 
        n_sta_low = n_sta_up 
    n_sta = n_sta_up+n_sta_low
    
    transition_m = np.zeros((n_sta,n_sta))
    
    states = det_state_2ways(1,-1, n_sta_up=n_sta_up, n_sta_low=n_sta_low) 
    for fluxs in wind_fluxs:
        add_transitions(fluxs, transition_m, states)
    
    for i in range(n_sta):
        suma_i = 0
        for j in range(n_sta):
            transition_m[i,j] +=1 #priors
            suma_i += transition_m[i,j]
        transition_m[i] = transition_m[i]/suma_i #normalize
    return transition_m


In [ ]:
%%time
listita = [[10,10],[10,15],[10,20],[10,30],[15,10],[15,15],[15,20],[15,30],[5,10],[5,15],[5,20],[5,30]]
for values in listita:
    final_npy = []
    i=0
    for lc_our_detrend in total_time_list:   
        print ("recuperando curva",i+1)
        fluxs = prepare_lc(lc_our_detrend) #divide by min
        #plt.figure(figsize=(15,10))
        #plt.plot(fluxs)
        #plt.show()
        lc_tramos =  extract_nans(fluxs) #extract nans  
        transition_m = manual_HMM(lc_tramos, n_sta_up=values[0], n_sta_low=values[1]) #si se dejan 5 arriba y 10 abajo cambia harto...

        i+=1
        #plt.imshow(transition_m, cmap='Blues', vmin=0, vmax=1) #el plot se ve raro por esto..
        #plt.show()
        final_npy.append(transition_m)
        
    final_npy = np.asarray(final_npy)
    np.save('/work/work_teamEXOPLANET/MTF_gabo/npys/MTF_time_u%s_d%s.npy'%(str(values[0]),str(values[1])), final_npy)

recuperando curva 1
recuperando curva 2
recuperando curva 3
recuperando curva 4
recuperando curva 5
recuperando curva 6
recuperando curva 7
recuperando curva 8
recuperando curva 9
recuperando curva 10
recuperando curva 11
recuperando curva 12
recuperando curva 13
recuperando curva 14
recuperando curva 15
recuperando curva 16
recuperando curva 17
recuperando curva 18
recuperando curva 19
recuperando curva 20
recuperando curva 21
recuperando curva 22
recuperando curva 23
recuperando curva 24
recuperando curva 25
recuperando curva 26
recuperando curva 27
recuperando curva 28
recuperando curva 29
recuperando curva 30
recuperando curva 31
recuperando curva 32
recuperando curva 33
recuperando curva 34
recuperando curva 35
recuperando curva 36
recuperando curva 37
recuperando curva 38
recuperando curva 39
recuperando curva 40
recuperando curva 41
recuperando curva 42
recuperando curva 43
recuperando curva 44
recuperando curva 45
recuperando curva 46
recuperando curva 47
recuperando curva 48
r

recuperando curva 379
recuperando curva 380
recuperando curva 381
recuperando curva 382
recuperando curva 383
recuperando curva 384
recuperando curva 385
recuperando curva 386
recuperando curva 387
recuperando curva 388
recuperando curva 389
recuperando curva 390
recuperando curva 391
recuperando curva 392
recuperando curva 393
recuperando curva 394
recuperando curva 395
recuperando curva 396
recuperando curva 397
recuperando curva 398
recuperando curva 399
recuperando curva 400
recuperando curva 401
recuperando curva 402
recuperando curva 403
recuperando curva 404
recuperando curva 405
recuperando curva 406
recuperando curva 407
recuperando curva 408
recuperando curva 409
recuperando curva 410
recuperando curva 411
recuperando curva 412
recuperando curva 413
recuperando curva 414
recuperando curva 415
recuperando curva 416
recuperando curva 417
recuperando curva 418
recuperando curva 419
recuperando curva 420
recuperando curva 421
recuperando curva 422
recuperando curva 423
recuperand

recuperando curva 752
recuperando curva 753
recuperando curva 754
recuperando curva 755
recuperando curva 756
recuperando curva 757
recuperando curva 758
recuperando curva 759
recuperando curva 760
recuperando curva 761
recuperando curva 762
recuperando curva 763
recuperando curva 764
recuperando curva 765
recuperando curva 766
recuperando curva 767
recuperando curva 768
recuperando curva 769
recuperando curva 770
recuperando curva 771
recuperando curva 772
recuperando curva 773
recuperando curva 774
recuperando curva 775
recuperando curva 776
recuperando curva 777
recuperando curva 778
recuperando curva 779
recuperando curva 780
recuperando curva 781
recuperando curva 782
recuperando curva 783
recuperando curva 784
recuperando curva 785
recuperando curva 786
recuperando curva 787
recuperando curva 788
recuperando curva 789
recuperando curva 790
recuperando curva 791
recuperando curva 792
recuperando curva 793
recuperando curva 794
recuperando curva 795
recuperando curva 796
recuperand

recuperando curva 1119
recuperando curva 1120
recuperando curva 1121
recuperando curva 1122
recuperando curva 1123
recuperando curva 1124
recuperando curva 1125
recuperando curva 1126
recuperando curva 1127
recuperando curva 1128
recuperando curva 1129
recuperando curva 1130
recuperando curva 1131
recuperando curva 1132
recuperando curva 1133
recuperando curva 1134
recuperando curva 1135
recuperando curva 1136
recuperando curva 1137
recuperando curva 1138
recuperando curva 1139
recuperando curva 1140
recuperando curva 1141
recuperando curva 1142
recuperando curva 1143
recuperando curva 1144
recuperando curva 1145
recuperando curva 1146
recuperando curva 1147
recuperando curva 1148
recuperando curva 1149
recuperando curva 1150
recuperando curva 1151
recuperando curva 1152
recuperando curva 1153
recuperando curva 1154
recuperando curva 1155
recuperando curva 1156
recuperando curva 1157
recuperando curva 1158
recuperando curva 1159
recuperando curva 1160
recuperando curva 1161
recuperando

recuperando curva 1476
recuperando curva 1477
recuperando curva 1478
recuperando curva 1479
recuperando curva 1480
recuperando curva 1481
recuperando curva 1482
recuperando curva 1483
recuperando curva 1484
recuperando curva 1485
recuperando curva 1486
recuperando curva 1487
recuperando curva 1488
recuperando curva 1489
recuperando curva 1490
recuperando curva 1491
recuperando curva 1492
recuperando curva 1493
recuperando curva 1494
recuperando curva 1495
recuperando curva 1496
recuperando curva 1497
recuperando curva 1498
recuperando curva 1499
recuperando curva 1500
recuperando curva 1501
recuperando curva 1502
recuperando curva 1503
recuperando curva 1504
recuperando curva 1505
recuperando curva 1506
recuperando curva 1507
recuperando curva 1508
recuperando curva 1509
recuperando curva 1510
recuperando curva 1511
recuperando curva 1512
recuperando curva 1513
recuperando curva 1514
recuperando curva 1515
recuperando curva 1516
recuperando curva 1517
recuperando curva 1518
recuperando

...

recuperando curva 592
recuperando curva 593
recuperando curva 594
recuperando curva 595
recuperando curva 596
recuperando curva 597
recuperando curva 598
recuperando curva 599
recuperando curva 600
recuperando curva 601
recuperando curva 602
recuperando curva 603
recuperando curva 604
recuperando curva 605
recuperando curva 606
recuperando curva 607
recuperando curva 608
recuperando curva 609
recuperando curva 610
recuperando curva 611
recuperando curva 612
recuperando curva 613
recuperando curva 614
recuperando curva 615
recuperando curva 616
recuperando curva 617
recuperando curva 618
recuperando curva 619
recuperando curva 620
recuperando curva 621
recuperando curva 622
recuperando curva 623
recuperando curva 624
recuperando curva 625
recuperando curva 626
recuperando curva 627
recuperando curva 628
recuperando curva 629
recuperando curva 630
recuperando curva 631
recuperando curva 632
recuperando curva 633
recuperando curva 634
recuperando curva 635
recuperando curva 636
recuperand

recuperando curva 965
recuperando curva 966
recuperando curva 967
recuperando curva 968
recuperando curva 969
recuperando curva 970
recuperando curva 971
recuperando curva 972
recuperando curva 973
recuperando curva 974
recuperando curva 975
recuperando curva 976
recuperando curva 977
recuperando curva 978
recuperando curva 979
recuperando curva 980
recuperando curva 981
recuperando curva 982
recuperando curva 983
recuperando curva 984
recuperando curva 985
recuperando curva 986
recuperando curva 987
recuperando curva 988
recuperando curva 989
recuperando curva 990
recuperando curva 991
recuperando curva 992
recuperando curva 993
recuperando curva 994
recuperando curva 995
recuperando curva 996
recuperando curva 997
recuperando curva 998
recuperando curva 999
recuperando curva 1000
recuperando curva 1001
recuperando curva 1002
recuperando curva 1003
recuperando curva 1004
recuperando curva 1005
recuperando curva 1006
recuperando curva 1007
recuperando curva 1008
recuperando curva 1009


recuperando curva 1323
recuperando curva 1324
recuperando curva 1325
recuperando curva 1326
recuperando curva 1327
recuperando curva 1328
recuperando curva 1329
recuperando curva 1330
recuperando curva 1331
recuperando curva 1332
recuperando curva 1333
recuperando curva 1334
recuperando curva 1335
recuperando curva 1336
recuperando curva 1337
recuperando curva 1338
recuperando curva 1339
recuperando curva 1340
recuperando curva 1341
recuperando curva 1342
recuperando curva 1343
recuperando curva 1344
recuperando curva 1345
recuperando curva 1346
recuperando curva 1347
recuperando curva 1348
recuperando curva 1349
recuperando curva 1350
recuperando curva 1351
recuperando curva 1352
recuperando curva 1353
recuperando curva 1354
recuperando curva 1355
recuperando curva 1356
recuperando curva 1357
recuperando curva 1358
recuperando curva 1359
recuperando curva 1360
recuperando curva 1361
recuperando curva 1362
recuperando curva 1363
recuperando curva 1364
recuperando curva 1365
recuperando

recuperando curva 1680
recuperando curva 1681
recuperando curva 1682
recuperando curva 1683
recuperando curva 1684
recuperando curva 1685
recuperando curva 1686
recuperando curva 1687
recuperando curva 1688
recuperando curva 1689
recuperando curva 1690
recuperando curva 1691
recuperando curva 1692
recuperando curva 1693
recuperando curva 1694
recuperando curva 1695
recuperando curva 1696
recuperando curva 1697
recuperando curva 1698
recuperando curva 1699
recuperando curva 1700
recuperando curva 1701
recuperando curva 1702
recuperando curva 1703
recuperando curva 1704
recuperando curva 1705
recuperando curva 1706
recuperando curva 1707
recuperando curva 1708
recuperando curva 1709
recuperando curva 1710
recuperando curva 1711
recuperando curva 1712
recuperando curva 1713
recuperando curva 1714
recuperando curva 1715
recuperando curva 1716
recuperando curva 1717
recuperando curva 1718
recuperando curva 1719
recuperando curva 1720
recuperando curva 1721
recuperando curva 1722
recuperando

recuperando curva 2037
recuperando curva 2038
recuperando curva 2039
recuperando curva 2040
recuperando curva 2041
recuperando curva 2042


In [ ]:
final_npy = np.asarray(final_npy)
np.save('/work/work_teamEXOPLANET/MTF_gabo/npys/MTF_time_u10_d10.npy', final_npy)

In [ ]:
!ls 